In [13]:
import torch
from simpletransformers.ner import NERModel, NERArgs

_NER_TAGS = ["O", "B_D", "B_E", "I_D", "I_E"]
ner_args = NERArgs()
ner_args.max_seq_length = 256

test_ner = NERModel("bert", 'outputs/best_model', args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS)

In [14]:
def tag_html_format(pred_list):
    LIST_TAGS = ['delete', 'edit', 'o']
    REPRESEN_TAGS = ['d', 'e', 'O']
    text_convert = ''
    prev_tag = ''
    trigger_tag = False 


    for word, label in pred_list:
        tags = label.split('-')
        next_tag = tags[0] if len(tags) == 1 else tags[1]
        
        if prev_tag != next_tag:
            if prev_tag:
                text_convert += '</' + html_tag + '>'

            html_tag = LIST_TAGS[REPRESEN_TAGS.index(next_tag)]
            prev_tag = next_tag
            trigger_tag = not(trigger_tag)

            if trigger_tag:
                text_convert += '<' + html_tag + '>'
            else:
                text_convert += '<' + html_tag + '>'

        text_convert += word
    text_convert += '</'+ LIST_TAGS[REPRESEN_TAGS.index(prev_tag)]+'>'
    text_convert = text_convert.replace('<o>', '').replace('</o>', '')
            
    return text_convert

In [15]:
def predict_text(text):
    predict_text = []
    temp = []
    text_token = word_tokenize(text)
    text_token = [word if word.strip() != '' else '_' for word in text_token]
    predictions, raw_outputs = test_ner.predict([text_token], split_on_space=False)
    # convert [{}] to [()] and convert B_D to B-d
    for pred_dict in predictions[0]:
        for word, label in pred_dict.items():
             label = label.capitalize().replace('_', '-')
             predict_text.append((word, label))
    predict_text = [(word, label) if word != '_' else (' ', label) for word, label in predict_text]
    return predict_text, tag_html_format(predict_text)

In [18]:
from pythainlp.tokenize import word_tokenize

text = 'ในช่วงต้นเดือนพฤศจิกายน 2014 "รายการราคา" ที่น่ากลัวสำหรับสตรียาซิดีและคริสเตียนปรากฏขึ้นทางออนไลน์'

pred_text, pred_tag = predict_text(text)
print(pred_text)
print("----")
print(pred_tag)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[('ใน', 'O'), ('ช่วง', 'O'), ('ต้นเดือน', 'O'), ('พฤศจิกายน', 'O'), (' ', 'O'), ('2014', 'O'), (' ', 'O'), ('"', 'O'), ('รายการ', 'O'), ('ราคา', 'O'), ('"', 'O'), (' ', 'O'), ('ที่', 'O'), ('น่ากลัว', 'O'), ('สำหรับ', 'O'), ('สตรี', 'O'), ('ยา', 'O'), ('ซิ', 'O'), ('ดี', 'O'), ('และ', 'O'), ('คริสเตียน', 'O'), ('ปรากฏ', 'O'), ('ขึ้น', 'O'), ('ทาง', 'O'), ('ออนไลน์', 'O')]
----
ในช่วงต้นเดือนพฤศจิกายน 2014 "รายการราคา" ที่น่ากลัวสำหรับสตรียาซิดีและคริสเตียนปรากฏขึ้นทางออนไลน์


In [17]:
text = "ทางตะวันออกของแร็งส์ กองทัพที่สี่ของฝรั่งเศสได้เตรียมการป้องกันอย่างเชี่ยวชาญในเชิงลึกเพื่อตอบโต้การทิ้งระเบิดที่รุนแรงและการแทรกซึมของทหารราบ"
text_token = word_tokenize(text)
print(text_token)

['ทาง', 'ตะวันออก', 'ของ', 'แร็งส์', ' ', 'กองทัพ', 'ที่สี่', 'ของ', 'ฝรั่งเศส', 'ได้', 'เตรียม', 'การป้องกัน', 'อย่าง', 'เชี่ยวชาญ', 'ใน', 'เชิง', 'ลึก', 'เพื่อ', 'ตอบโต้', 'การ', 'ทิ้ง', 'ระเบิด', 'ที่', 'รุนแรง', 'และ', 'การ', 'แทรกซึม', 'ของ', 'ทหารราบ']


In [ ]:
การชุมนุมของแนวร่วมประชาธิปไตยต่อต้านเผด็จการแห่งชาติ พ.ศ. 2553 เริ่มตั้งแต่วันที่ 12 มีนาคม พ.ศ. 2553 มีเป้าหมายเรียกร้องให้อภิสิทธิ์ เวชชาชีวะ นายกรัฐมนตรีไทย ประกาศการยุบสภาผู้แทนราษฎร และจัดการเลือกตั้งใหม่ ต่อมา มีการสร้างสถานการณ์โดยกลุ่มผู้ชุมนุม จนทำให้มีผู้เสียชีวิต 91 ศพ